# End to End Industrial IoT (IIoT) on Azure Databricks 
## Part 3 - Model Deployment and Inference with Azure ML

In the original Databricks exercise, these steps were in Part 2. However the method used for deployment has been deprecated. One of the new methods is used for deployment of the life and power models to web service endpoints. Code is then executed to invoke the life and power models to make predictions. These predictors are then used to optimize the Wind Turbine RPM to maximize power while controlling costs.

In [0]:
# Widgets containing important user IDs, names and keys. 
dbutils.widgets.text("Subscription ID","","Subscription ID")
dbutils.widgets.text("Resource Group","","Resource Group")
dbutils.widgets.text("Region","","Region")
dbutils.widgets.text("Workspace", "", "Workspace")
dbutils.widgets.text("Tenant", "", "Tenant")
dbutils.widgets.text("Client ID", "", "Client ID")
dbutils.widgets.text("Client Secret", "", "Client Secret")
dbutils.widgets.text("Life API Key", "", "Life API Key")
dbutils.widgets.text("Power API Key", "", "Power API Key")


## Environment Setup

The pre-requisites are listed below:

### Azure Services Required
* ADLS Gen 2 Storage account with a container called `iot`
* Azure Machine Learning Workspace called `iot`

### Azure Databricks Configuration Required
* 3-node (min) Databricks Cluster running **DBR 10.4 ML+** and the following libraries:
 * **MLflow[AzureML]** - PyPI library `azureml-mlflow` version
 * **Azure Event Hubs Connector for Databricks** - Maven coordinates `com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.21`
 * **Azure ML Package client library for Python** - PyPI library `azure-ai-ml` version 1.5.0
 * **MLflow** - PyPI library `mlflow` version 1.30.0
 * **Azure Machine Learning core packages, modules, and classes** - PyPI library `azureml.core` version 1.47.0
 * **

* The following notebook widgets populated:
 * `Subscription ID` - subscription ID of your Azure ML Workspace
 * `Resource Group` - resource group name of your Azure ML Workspace
 * `Region` - Azure region of your Azure ML Workspace
 * `Workspace` - Name of the Azure Machine Learning Workspace
 * `Tenant` - Active Directory Tenant ID for Service Principal
 * `Client ID` - Active Directory Client ID for Service Principal
 * `Client Secret` - Active Directory Client Secret for Service Principal
 * `Life API Key` - Life Prediction service API key
 * `Power API Key` - Power Prediction service API key
 * **

* **Part 1 Notebook Run to generate and process the data**. 
* Ensure the following tables have been created:
 * **turbine_maintenance** - Maintenance dates for each Wind Turbine
 * **turbine_power** - Hourly power output for each Wind Turbine
 * **turbine_enriched** - Hourly turbine sensor readinigs (RPM, Angle) enriched with weather readings (temperature, wind speed/direction, humidity)
 * **gold_readings** - Combined view containing all 3 tables
* **
* **Part 2 Notebook Run to train models and create predictions table**. 
* Ensure the following models have been created:
 * **power_prediction** - Predict power 6 hours ahead for each Wind Turbine
 * **life_prediction** - Predict the remaining life of a turbine before maintenance is required
* Ensure the following tables have been created:
 * **turbine_power_predictions**
 * **turbine_life_predictions**

In [0]:
# Verify azureml.core is installed
!pip show azureml.core

In [0]:
# Verify azureml-mlflow is installed
!pip show azureml-mlflow

In [0]:
# Verify azure-ai-ml is installed
!pip show azure-ai-ml

In [0]:
# This is automatically installed
!pip show azure.core

In [0]:
# Import libraries and print out the versions
import xgboost as xgb
import azureml.mlflow
import azureml.core
import mlflow
import pandas as pd
import sklearn
import numpy as np
import matplotlib


# cell to check for versions
print("XGBoost: {}".format(xgb.__version__))
print("Pandas: {}".format(np.__version__))
print("MLFlow: {}".format(mlflow.__version__))
print("Matplotlib: {}".format(matplotlib.__version__))
print("Scikit-Learn: {}".format(sklearn.__version__))
print("azureml-mlflow: {}".format(azureml.mlflow.__version__))
print("azureml.core: {}".format(azureml.core.__version__))
print("NumPy: {}".format(np.__version__))

In [0]:
# If there is an issue with typing_extensions, uncomment out this cell and run it
# from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow
# import typing_extensions
# from importlib import reload
# reload(typing_extensions)

In [0]:
# This cell needs to run each time
# Set important id's and keys from widgets

subscription_id = dbutils.widgets.get("Subscription ID")
workspace = dbutils.widgets.get("Workspace")
resource_group = dbutils.widgets.get("Resource Group")
tenant_id = dbutils.widgets.get("Tenant")
client_id = dbutils.widgets.get("Client ID")
client_secret = dbutils.widgets.get("Client Secret")
life_api_key = dbutils.widgets.get("Life API Key")
power_api_key = dbutils.widgets.get("Power API Key")


## BLOB_CONTAINER_NAME = "iot"
# set the ADLS KEY in the spark configuration
## spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", adls_key)

# Setup storage locations for all data
## ROOT_PATH = f"abfss://iot@{storage_account}.dfs.core.windows.net/"

# Pyspark and ML Imports
import os, json, requests
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
import numpy as np 
import pandas as pd
import xgboost as xgb
import mlflow.xgboost
import mlflow.azureml
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice, Webservice
import random, string

# Random String generator for ML models served in AzureML
random_string = lambda length: ''.join(random.SystemRandom().choice(string.ascii_lowercase) for _ in range(length))

In [0]:
# This cell needs to run each time
# This is for service principal ml-auth
# these values will be picked up by DefaultAzureCredential
import os

os.environ["AZURE_TENANT_ID"] = tenant_id
os.environ["AZURE_CLIENT_ID"] = client_id
os.environ["AZURE_CLIENT_SECRET"] = client_secret

## Configuring models' registry

MLflow allows you to segregate the instance where experiments are being tracked from the instance where models' are being tracked (or registered). The first one is referred to **Tracking URI** while the second one is referred as **Registry URI**. By default, both of them are set to the same value, and in Azure Databricks, both of them are set to "databricks" meaning that tracking and model registries will happen inside of the MLflow instance that Databricks runs for you.

We are going to track the experiments in Azure Databricks, but change so that model registries will be held in Azure ML. This will allow us to manage the model's lifecycle - including deployments - in Azure ML.

First we create and connect to an MLClient for our workspace, then we get the Azure MLflow tracking URI which will be used to set the Databricks MLflow registry URI.

In [0]:
# This cell needs to be executed each time
# From here, each cell requires the previous cell to be executed...you can't just skip any group of cells
# Note that this cell outputs sensitive parameter values to the notebook if you print(ml_client)
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# This will show key ID's in the results - so leave it commented unless you want to verify
# print(ml_client)

In [0]:
# Get the Azure Tracking URI which we will use to set the Databricks Resgistry URI
azureml_tracking_uri = ml_client.workspaces.get(
    ml_client.workspace_name
).mlflow_tracking_uri
print(azureml_tracking_uri)

In [0]:
# set the tracking URI for Databricks mlflow
# remember that mlflow is for Databricks and azureml-mlflow is for Azure ML
import mlflow

mlflow.set_registry_uri(azureml_tracking_uri)

## Registering the model in Azure ML

So far, our model is trained and tracked inside of the MLflow instance in Azure Databricks. Now we want to register this model in Azure ML to manage the life cicle there. However, if we try to register the model as we usually do using the syntax `mlflow.register_model(model_uri=f"runs:/{run.info.run_id}/model").` you will found an error. The reason why this is happening is related to where runs are being stored.

Right now runs are being stored in Azure Databricks and models in Azure ML. If you try to create a registered model from a Run, Azure ML doesn't have any way to guess how to get access to the runs, that are stored in a different service. because of that, you can't use `runs:/` URI for registering models.

To overcome this limitation, you have to register the model from the artifacts themselfs, which you can achieve by first downloading them.

In [0]:
#download artifacts for model runs to use in registering with Azure ML
import mlflow

#You need to lookup the run id's in Databricks and paste them here 
#paste in the life prediction run id
life_prediction_run_id = "" 

#paste in the power prediction run id
power_prediction_run_id = "" 

# access the tracking client
client = mlflow.tracking.MlflowClient()

# first get it from looking up manually 
life_model_path = client.download_artifacts(life_prediction_run_id, path="model")
power_model_path = client.download_artifacts(power_prediction_run_id, path="model")

print(life_model_path)
print(power_model_path)

In [0]:
# Register the two models
mlflow.register_model(
    model_uri=f"file://{life_model_path}", name="life_prediction"
)

mlflow.register_model(
    model_uri=f"file://{power_model_path}", name="power_prediction"
)

Notice in the code above how the protocol is now `file://` instead of `runs:/`.
Now look for model in Azure ML Studio in Models

## Deploy the model as an online endpoint

Now deploy your machine learning model as a web service in the Azure cloud, an [`online endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

To deploy a machine learning service, you usually need:

* The model assets (file, metadata) that you want to deploy. You've already registered these assets in Azure ML after downloading from Azure Databricks.
* Some code to run as a service. The code executes the model on a given input request. This entry script receives data submitted to a deployed web service and passes it to the model, then returns the model's response to the client. The script is specific to your model. The entry script must understand the data that the model expects and returns. With an MLFlow model, as in this tutorial, this script is automatically created for you. Samples of scoring scripts can be found [here](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk/endpoints/online).

## Create a new online endpoint

Now that you have a registered model for and an inference script, it's time to create your online endpoint for both life and power. The endpoint name needs to be unique in the entire Azure region. For this tutorial, you'll create a unique name using [`UUID`](https://en.wikipedia.org/wiki/Universally_unique_identifier#:~:text=A%20universally%20unique%20identifier%20(UUID,%2C%20for%20practical%20purposes%2C%20unique.).

In [0]:
# create unique names for our endpoints
import uuid

# Creating a unique name for the life prediction endpoint
life_online_endpoint_name = "life-endpoint-" + str(uuid.uuid4())[:8]

# Creating a unique name for the power prediction endpoint
power_online_endpoint_name = "power-endpoint-" + str(uuid.uuid4())[:8]

In [0]:
# create the endpoints
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
life_endpoint = ManagedOnlineEndpoint(
    name=life_online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "wind turbine rul",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

life_endpoint = ml_client.online_endpoints.begin_create_or_update(life_endpoint).result()

# create an online endpoint
power_endpoint = ManagedOnlineEndpoint(
    name=power_online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "wind turbine power",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

power_endpoint = ml_client.online_endpoints.begin_create_or_update(power_endpoint).result()

print(f"Endpoint {life_endpoint.name} provisioning state: {life_endpoint.provisioning_state}")
print(f"Endpoint {power_endpoint.name} provisioning state: {power_endpoint.provisioning_state}")


## Once you've created an endpoint, you can retrieve it as below:

In [0]:
# verify that we have the correct ml_client
life_endpoint = ml_client.online_endpoints.get(name=life_online_endpoint_name)

print(
    f'Endpoint "{life_endpoint.name}" with provisioning state "{life_endpoint.provisioning_state}" is retrieved'
)

power_endpoint = ml_client.online_endpoints.get(name=power_online_endpoint_name)

print(
    f'Endpoint "{power_endpoint.name}" with provisioning state "{power_endpoint.provisioning_state}" is retrieved'
)

## Deploy the model to the endpoint

Once the endpoint is created, deploy the model with the entry script. Each endpoint can have multiple deployments. Direct traffic to these deployments can be specified using rules. Here you'll create a single deployment that handles 100% of the incoming traffic. We have chosen a color name for the deployment, for example, *blue*, *green*, *red* deployments, which is arbitrary.

You can check the **Models** page on Azure ML studio, to identify the latest version of your registered model. Alternatively, the code below will retrieve the latest version number for you to use.

In [0]:
# 2 model names are life_prediction and power_prediction
# Let's pick the latest version of the model
latest_life_model_version = max(
    [int(m.version) for m in ml_client.models.list(name="life_prediction")]
)

latest_power_model_version = max(
    [int(m.version) for m in ml_client.models.list(name="power_prediction")]
)

print(f'Life model version is "{latest_life_model_version}"')
print(f'Power model version is "{latest_power_model_version}"')

Deploy the latest version of the model.  

> [!NOTE]
> Expect this deployment to take approximately 12 to 18 minutes.

In [0]:
# picking the model to deploy. Here we use the latest version of our registered model
life_model = ml_client.models.get(name="life_prediction", version=latest_life_model_version)


# create an online deployment. Change this type to smaller and see how to track running instances/endpoints
# changed from STANDARD_DS3_V2
blue_life_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=life_online_endpoint_name,
    model=life_model,
    instance_type="STANDARD_F4S_V2",
    instance_count=1,
)

blue_life_deployment = ml_client.begin_create_or_update(blue_life_deployment).result()

# picking the model to deploy. Here we use the latest version of our registered model
power_model = ml_client.models.get(name="power_prediction", version=latest_power_model_version)


# create an online deployment. Change this type to smaller and see how to track running instances/endpoints
# changed from STANDARD_DS3_V2
blue_power_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=power_online_endpoint_name,
    model=power_model,
    instance_type="STANDARD_F4S_V2",
    instance_count=1,
)

blue_power_deployment = ml_client.begin_create_or_update(blue_power_deployment).result()
print(blue_life_deployment)
print(blue_power_deployment)

## Call with code suggested in "Consume" tab of ML Studio adapted from original Wind Turbine Part 2

In [0]:
import urllib.request
import json
import os
import ssl

#uri's are blue_power_deployment, blue_life_deployment
#keys are life_api_key, power_api_key

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

def score_data(uri, key, data):

  if not key:
    raise Exception("A key should be provided to invoke the endpoint")
  
  # The azureml-model-deployment header will force the request to go to a specific deployment.
  # Remove this header to have the request observe the endpoint traffic rules
  headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ key), 'azureml-model-deployment': 'blue' }
  body = str.encode(json.dumps(data))
  req = urllib.request.Request(uri, body, headers)

  try:
      response = urllib.request.urlopen(req)

      result = response.read()
      #print(result)
      return(result)
  except urllib.error.HTTPError as error:
      print("The request failed with status code: " + str(error.code))

      # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
      print(error.info())
      print(error.read().decode("utf8", 'ignore'))
      # is there something better to return ? 
      return("")

 

In [0]:
#Test calls to service
#uri's are blue_power_deployment, blue_life_deployment
#keys are life_api_key, power_api_key

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "input_data": {
    "columns": [
      "angle",
      "rpm",
      "temperature",
      "humidity",
      "windspeed",
      "power",
      "age"
    ],
    "index": [1,7],
    "data": [[8,6,25,50,5,150,10]]
  }
}


# get keys in case they were recently entered
life_api_key = dbutils.widgets.get("Life API Key")
power_api_key = dbutils.widgets.get("Power API Key")

# we get the scoring_uri from each endpoint that we saved above when endpoints were created : power_endpoint, life_endpoint
print(f'Current Operating Parameters: {data}')
print(f'Predicted power (in kwh) from model: {score_data(power_endpoint.scoring_uri, power_api_key, data)}')
print(f'Predicted remaining life (in days) from model: {score_data(life_endpoint.scoring_uri, life_api_key, data)}')   

### Asset Optimization
We can now identify the optimal operating conditions for maximizing power output while also maximizing asset useful life. 

\\(Revenue = Price\displaystyle\sum_1^{365} Power_t\\)

\\(Revenue = {365 * } Price \displaystyle\sum_1^{24} Power_t \\)

\\(Cost = {365 \over Life_{rpm}} Price \displaystyle\sum_1^{24} Power_t \\)

\\(Profit = Revenue - Cost\\)

\\(Power_t\\) and \\(Life\\) will be calculated by scoring many different RPM values in AzureML. The results can be visualized to identify the RPM that yields the highest profit.

In [0]:
import urllib.request
import json
import os
import ssl

#uri's are blue_power_deployment, blue_life_deployment
#keys are life_api_key, power_api_key

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

def create_packet (angle, rpm, temp, hum, wind, power, age):
  # Request data goes here
  # The example below assumes JSON formatting which may be updated
  # depending on the format your endpoint expects.
  # More information can be found here:
  # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
  data =  {
    "input_data": {
      "columns": [
        "angle",
        "rpm",
        "temperature",
        "humidity",
        "windspeed",
        "power",
        "age"
      ],
      "index": [1,7],
      "data": [[angle, rpm, temp, hum, wind, power, age]]
    }
  }

  return(data)


# Iterate through 14 different RPM configurations and capture the predicted power and remaining life at each RPM
results = []
for rpm in range(1,15):
  data = create_packet(8,rpm,25,50,5,150,10)
  expected_power = score_data(power_endpoint.scoring_uri, power_api_key, data)[0]
  data['power'] = expected_power
  expected_life = -score_data(life_endpoint.scoring_uri, life_api_key, data)[0]
  results.append((rpm, expected_power, expected_life))
  
# Calculate the Revenue, Cost and Profit generated for each RPM configuration
optimization_df = pd.DataFrame(results, columns=['RPM', 'Expected Power', 'Expected Life'])
optimization_df['Revenue'] = optimization_df['Expected Power'] * 24 * 365
optimization_df['Cost'] = optimization_df['Expected Power'] * 24 * 365 / optimization_df['Expected Life']
optimization_df['Profit'] = optimization_df['Revenue'] + optimization_df['Cost']

display(optimization_df)

##### The optimal operating parameters for WindTurbine-1 given the specified weather conditions is 11 RPM for generating a maximum profit of $1.4M! Your results may vary due to the random nature of the sensor readings.

You can view your model, it's deployments and URL endpoints by navigating to https://ml.azure.com/.

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/iiot_azureml.gif" width=800>

## Clean up resources - delete endpoints

If you're not going to use the endpoint, delete it to stop using the resource.  Make sure no other deployments are using an endpoint before you delete it.


> [!NOTE]
> Expect this step to take approximately 6 to 8 minutes.

In [0]:
print(life_online_endpoint_name)
print(power_online_endpoint_name)

In [0]:
# Delete the endpoints running our models as services
ml_client.online_endpoints.begin_delete(name=life_online_endpoint_name)
ml_client.online_endpoints.begin_delete(name=power_online_endpoint_name)